### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [2]:
!pip install implicit
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

# from lightfm.datasets import fetch_movielens

     |████████████████████████████████| 1.1MB 9.9MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406380 sha256=b97144fa211b099422f826bf7835a8c61c01672ab1242e6335b163920c638cc4
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
github_ratings = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/ratings.dat'
ratings = pd.read_csv(github_ratings, delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
github_movies = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/movies.dat'
movie_info = pd.read_csv(github_movies, delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='ISO-8859-1')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(1, model)

['0    Toy Story (1995)',
 '346    Client, The (1994)',
 '120    Boomerang (1992)',
 '3419    Hideous Sun Demon, The (1959)',
 '3626    Toxic Avenger Part III: The Last Temptation of...',
 '2327    Shakespeare in Love (1998)',
 '279    Nell (1994)',
 '1556    Conspiracy Theory (1997)',
 '2546    My Science Project (1985)',
 '2334    First Blood (1982)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [17]:
get_recommendations(4, model)

['2275    Runaway Train (1985)',
 '117    Steal Big, Steal Little (1995)',
 '3012    Sleepy Hollow (1999)',
 '3731    Criminal Lovers (Les Amants Criminels) (1999)',
 '750    Stalingrad (1993)',
 '66    Two Bits (1995)',
 '3250    Judy Berlin (1999)',
 '536    Sliver (1993)',
 '2229    Strangeland (1998)',
 '2334    First Blood (1982)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [18]:
from sklearn.base import TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split

In [19]:
class SGD:
    def __init__(self, features=64, iterations=int(5e6), learning_rate=0.01, matrix_reg=0.01, bias_reg=0.01):
        self.features = features
        self.iterations = iterations
        self.learning_rate = learning_rate
        
        self.user_reg = matrix_reg
        self.item_reg = matrix_reg
        self.user_bias_reg = bias_reg
        self.item_bias_reg = bias_reg


    def fit(self, df=ratings):
        self.df = df
        self.users = self.df['user_id'].max()
        self.items = self.df['movie_id'].max()
        self.avg = self.df['rating'].mean()

        self.U = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
        self.I = np.random.uniform(0, 1 / np.sqrt(self.features), (self.items, self.features)) 

        self.user_bias = np.zeros((self.users, 1))
        self.item_bias = np.zeros((self.items, 1))
    
        for _ in range(self.iterations):
            i, j, actual = map(int, df.sample().to_numpy().ravel())
            i -= 1
            j -= 1
            
            error = (self.U[i] @ self.I[j].T + self.user_bias[i] + self.item_bias[j].T + self.avg) - actual
            
            self.U[i] -= self.learning_rate * (error * self.I[j]   + self.user_reg * self.U[i])
            self.I[j] -= self.learning_rate * (error * self.U[i].T + self.item_reg * self.I[j])
            self.user_bias[i] -= self.learning_rate * (error + self.user_bias_reg * self.user_bias[i])
            self.item_bias[j] -= self.learning_rate * (error + self.item_bias_reg * self.item_bias[j])
        
        return self

    
    def similar_items(self, i, n=10):
        metric = lambda j: np.linalg.norm(self.I[i] - self.I[j])
        distances = [(j, metric(j)) for j in range(len(self.I))]
        return sorted(distances, key=lambda pair: pair[1])[:n]

In [20]:
%%time
sgd = SGD().fit()

KeyboardInterrupt: ignored

In [ ]:
get_similars(1, sgd)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [29]:
class ALS:
    def __init__(self, features=40, iterations=100, learning_rate=0.001, reg=1):
        self.features = features
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.reg = reg

    def fit(self, ratings=implicit_ratings):
        self.ratings = ratings
        self.users = self.ratings['user_id'].max()
        self.items = self.ratings['movie_id'].max()
        
        self.U = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
        self.I = np.random.uniform(0, 1 / np.sqrt(self.features), (self.items, self.features))

        for iteration in range(self.iterations):
            error = self.U @ self.I.T - user_item_csr


ALS().fit()

ValueError: ignored

In [46]:
implicit_ratings
user_item_csr[1, 3408]

1

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных